### Funcao para salvar os dados extraidos da web e colocar em um arquivo txt

In [ ]:
def salvar_no_arquivo(serial_number, status_text, warranty_text, produtoEspecText, arquivo_txt):
    """
    Salva as informações da máquina em um arquivo de texto.
    """
    try:
        with open(arquivo_txt, "a", encoding="utf-8") as log_file:
            log_file.write(f"Serial Number: {serial_number}\n")
            log_file.write(f"Status: {status_text}\n")
            log_file.write(f"Warranty: {warranty_text}\n")
            log_file.write(f"Especificações:\n{produtoEspecText}\n")
            log_file.write("----------------\n")
        print(f"Máquina registrada com sucesso no arquivo: {serial_number}")
    except Exception as e:
        print(f"Erro ao salvar no arquivo: {e}")

    except Exception as e:
        print(f"Erro ao processar máquina {serial_number}: {e}")


In [ ]:
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from openpyxl import load_workbook
from selenium import webdriver
from selenium.webdriver.chrome.service import Service


# Caminho para o ChromeDriver (não o navegador Chrome em si)
service = Service(r"C:\Users\rapha\OneDrive\Área de Trabalho\teste\chromedriver.exe")  # ou onde estiver seu chromedriver

# Inicializa o navegador
driver = webdriver.Chrome(service=service)


# Carregar o arquivo Excel
file_path = "serial_number.xlsx"
workbook = load_workbook(file_path)
sheet = workbook.active




In [ ]:
# Configurar o tempo de espera explícita
wait = WebDriverWait(driver, 20)

# Percorrer as células da coluna A
for row in range(2, sheet.max_row + 1):  # Começando da linha 2 para ignorar o cabeçalho
    serial_number = sheet[f'A{row}'].value

    # Acessar o site da Lenovo Warranty Lookup
    driver.get("https://pcsupport.lenovo.com/us/en/products/laptops-and-netbooks/thinkpad-x-series-laptops/thinkpad-x380-yoga/20lj/20lja029br/mp1en401/warranty/?linkTrack=Caps:Body_SearchProduct&searchType=6&keyWordSearch=mp1en401")
    time.sleep(5) 

    # Esperar o campo de input estar presente
    try:


        # Aguardar o campo correto para inserir o número de série
        input_field = wait.until(EC.presence_of_element_located((By.XPATH, '/html/body/div[2]/section[2]/div[1]/div[2]/div[2]/div[2]/span[1]/input[2]')))
        
        # Clicar no campo de entrada para habilitar a edição                
        input_field.click()
        # Tentar limpar o campo usando uma combinação de teclas
        input_field.send_keys(Keys.CONTROL + "a")  # Selecionar todo o texto no campo
        input_field.send_keys(Keys.DELETE)  # Deletar o texto selecionado
        print (serial_number)
        input_field.clear()
        input_field.send_keys(serial_number)
        input_field.send_keys(Keys.RETURN)

        time.sleep(5)  # Esperar a página carregar os resultados

        # Extrair o texto do campo de status da garantia
        status_xpath = '/html/body/div[2]/section[2]/div[1]/div[2]/div[2]/div[1]/h4'  # Ajuste este XPath conforme o conteúdo correto
        status_element = wait.until(EC.presence_of_element_located((By.XPATH, status_xpath)))
        status_text = status_element.text
        sheet[f'B{row}'] = status_text

        # Extrair o conteúdo do campo relacionado à garantia
        warranty_xpath = '//*[@id="app-psp-warranty"]/div[2]/div/div/div[1]/div/div[1]/div[1]/div/div/div[1]/div[1]/div/div[1]/div[2]/span[2]'  # Ajuste conforme o conteúdo correto
        warranty_element = wait.until(EC.presence_of_element_located((By.XPATH, warranty_xpath)))
        warranty_text = warranty_element.text
        sheet[f'C{row}'] = warranty_text
        
                
        # Encontra o elemento "Product home" pelo XPath e clica nele
        elemento = driver.find_element(By.XPATH, "/html/body/div[2]/section[2]/div[1]/div[3]/ul/li[1]/a/span[2]")
        elemento.click()
        
        time.sleep(5)
            
        # Encontra o elemento "Product View" pelo XPath e clica nele
        elemento = driver.find_element(By.XPATH, "//*[@id='app-psp-home']/div/div[2]/div[2]/section/div/div/div[4]/div[3]/div[2]/div/span")
        elemento.click()
        
        
        # Encontra o elemento pelo XPath e clica nele
        
        produtoEspec = '//*[@id="app-psp-home"]/div/div[2]/div[2]/section/div/div/div[4]/div[3]/div[2]/div/div'  # Ajuste conforme o conteúdo correto
        produtoEspecWait = wait.until(EC.presence_of_element_located((By.XPATH, produtoEspec)))
        produtoEspecText = produtoEspecWait.text

    except Exception as e:
        print(f"Erro na linha {row}: {e}")


    # Caminho para o arquivo de texto
    arquivo_txt = "maquinas_registradas.txt"


     # Salvar no arquivo texto
    salvar_no_arquivo(serial_number, status_text, warranty_text, produtoEspecText, arquivo_txt)
        
    
    # Salvar o arquivo Excel após cada iteração
    workbook.save(file_path)

# Fechar o navegador e encerrar a sessão
#driver.quit()


### Lê o arquivo de texto e separa os blocos de máquinas por modelo, salvando cada modelo em um arquivo separado.

In [1]:
import os
def processar_arquivo_txt(arquivo_entrada):

    try:
        with open(arquivo_entrada, "r", encoding="utf-8") as file:
            linhas = file.readlines()
        
        blocos = []
        bloco_atual = []
        
        for linha in linhas:
            if linha.strip() == "----------------":  # Indica separação de máquina
                if bloco_atual:  # Adiciona o bloco completo antes de resetar
                    blocos.append(bloco_atual)
                bloco_atual = []
            else:
                bloco_atual.append(linha.strip())

        if bloco_atual:  # Adiciona o último bloco
            blocos.append(bloco_atual)
        
        maquinas_por_modelo = {}

        for bloco in blocos:
            serial_number = bloco[0].split(": ")[1] if "Serial Number" in bloco[0] else "Desconhecido"
            status_text = next((linha.split(": ")[1] for linha in bloco if "Status" in linha), "Modelo Desconhecido")
            modelo_maquina = status_text.split(" - ")[0]  # Captura o modelo principal
            
            if modelo_maquina not in maquinas_por_modelo:
                maquinas_por_modelo[modelo_maquina] = []
            
            maquinas_por_modelo[modelo_maquina].append("\n".join(bloco) + "\n----------------\n")

        os.makedirs("arquivos", exist_ok=True)
    
        for modelo, conteudo in maquinas_por_modelo.items():
            nome_arquivo = os.path.join("arquivos", f"{modelo}.txt")
            with open(nome_arquivo, "w", encoding="utf-8") as arquivo_modelo:
                arquivo_modelo.writelines(conteudo)

        print("Arquivos gerados com sucesso!")
    
    except Exception as e:
        print(f"Erro ao processar o arquivo: {e}")


# Nome do arquivo de entrada contendo os dados
arquivo_entrada = "maquinas_registradas.txt"

# Executar a função
processar_arquivo_txt(arquivo_entrada)


Arquivos gerados com sucesso!


### Lê os aquivos no formato txt  e salva na planilha


In [2]:
import os
import re
import pandas as pd

mapeamento = {
    "Processor": "Processador",
    "Memory": "Memória",
    "Operating System": "Sistema Operacional",
    "Hard Drive": "Hard Drive",
    "Wired Network": "Rede cabeada",
    "Wireless Network": "Rede sem fio",
    "Ports": "Portas",
    "Camera": "Câmera",
    "Graphics": "Gráficos",
    "Monitor": "Monitores",
    "Form Factor": "Form Factor",
    "Included Warranty": "Garantia inclusa",
    "End of Service": "Fim de serviço"
}

def associar_categoria(descricao):
    for termo_eng, termo_pt in mapeamento.items():
        if descricao.strip().lower() in (termo_eng.lower(), termo_pt.lower()):
            return termo_pt
    return None

def processar_conteudo(texto):
    blocos = re.split(r"------------\s*", texto)
    dados_extraidos = []
    for bloco in blocos:
        if "Serial Number:" in bloco:
            maquina = {}
            serial = re.search(r"Serial Number:\s*(\S+)", bloco)
            if serial:
                maquina["Número de Série"] = serial.group(1)
            linhas = bloco.strip().splitlines()
            i = 0
            
            indices_categoria = []
            for idx, linha in enumerate(linhas):
                if associar_categoria(linha):
                    indices_categoria.append(idx)

            for i, idx in enumerate(indices_categoria):
                categoria = associar_categoria(linhas[idx])
                inicio_valor = idx + 1
                fim_valor = indices_categoria[i + 1] if i + 1 < len(indices_categoria) else len(linhas)
                valor = " ".join(l.strip() for l in linhas[inicio_valor:fim_valor]).strip()
                maquina[categoria] = valor

            if maquina:
                dados_extraidos.append(maquina)
    return dados_extraidos

modelos_validos = ["m900", "m910q", "t460", "t470", "t480", "x250", "x270", "Yoga 370", "X380 Yoga", "t14 gen 1", "t14 gen 2", "t14 gen 3", "t14 gen 4", "t14 gen 5", "M70q Gen 5"]
pasta = "./arquivos"
arquivo_excel = "maquinas.xlsx"

with pd.ExcelWriter(arquivo_excel, engine="openpyxl") as writer:
    for arquivo in os.listdir(pasta):
        if arquivo.endswith(".txt") and arquivo != "maquinas_registradas.txt":
            nome_base = arquivo.lower().replace(".txt", "")
            modelo_detectado = next((m for m in modelos_validos if m in nome_base), None)
            if modelo_detectado:
                caminho_arquivo = os.path.join(pasta, arquivo)
                with open(caminho_arquivo, "r", encoding="utf-8") as f:
                    conteudo = f.read()
                dados = processar_conteudo(conteudo)
                if dados:
                    df = pd.DataFrame(dados)
                    sheet_name = modelo_detectado.upper()
                    df.to_excel(writer, index=False, sheet_name=sheet_name)

    # Garante que pelo menos uma aba visível exista
    if not writer.sheets:
        pd.DataFrame([{"Aviso": "Nenhum arquivo de modelo válido foi processado."}])\
            .to_excel(writer, sheet_name="Aviso", index=False)

print(f"✅ Arquivo '{arquivo_excel}' criado com segurança.")


✅ Arquivo 'maquinas.xlsx' criado com segurança.


### Este bloco gera um arquivo .json com as especificações técnicas coletadas,
### estruturado para ser lido por um arquivo HTML dinâmico utilizado pela equipe de front-end

In [ ]:
import pandas as pd

# Carrega o arquivo Excel com todas as planilhas
arquivo_excel = pd.read_excel('maquinas.xlsx', sheet_name=None)

# Converte cada planilha em um dicionário
json_final = {}
for nome_sheet, df in arquivo_excel.items():
    json_final[nome_sheet] = df.to_dict(orient='records')



In [ ]:
# Salva em um arquivo JSON
import json
with open('saida.json', 'w', encoding='utf-8') as f:
    json.dump(json_final, f, ensure_ascii=False, indent=2)